In [ ]:
# 1. Install YOLO
!pip install -q ultralytics

# 2. FORCE REPAIR (Fixes the version conflicts)
!pip install "protobuf<=3.20.3" --force-reinstall
!pip install "numpy<2.0.0" --force-reinstall



In [1]:
import os
import cv2
import numpy as np
import pandas as pd
import glob
import gc
import tensorflow as tf
from scipy.ndimage import gaussian_filter1d

# --- 1. GPU CONFIGURATION ---
tf.config.set_soft_device_placement(True)

gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        print(f"🚀 GPU Active: {len(gpus)} device(s) found.")
    except RuntimeError as e:
        print(e)
else:
    print("⚠️ WARNING: No GPU found.")

from tensorflow.keras import layers, models, applications, optimizers
# Import YOLO LAST to avoid memory contention
from ultralytics import YOLO 

# --- 2. PATHS & SETTINGS ---
TRAIN_DIR = '/kaggle/input/pixel-play-26/Avenue_Corrupted-20251221T112159Z-3-001/Avenue_Corrupted/Dataset/training_videos'
TEST_DIR = '/kaggle/input/pixel-play-26/Avenue_Corrupted-20251221T112159Z-3-001/Avenue_Corrupted/Dataset/testing_videos'

IMG_SIZE = (128, 128)   # For Scene (Global)
CROP_SIZE = (64, 64)    # For Objects (Local)
BATCH_SIZE = 32         # Safe size for dual-model usage

2025-12-31 14:23:27.516172: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1767191007.714315     120 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1767191007.766145     120 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1767191008.212012     120 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1767191008.212071     120 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1767191008.212074     120 computation_placer.cc:177] computation placer alr

🚀 GPU Active: 1 device(s) found.
Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [2]:
print("⚙️ Loading Models...")

# 1. ResNet50 (Global Feature Extractor)
resnet = applications.ResNet50(weights='imagenet', include_top=False, pooling='avg')
resnet.trainable = False
print("✅ ResNet50 Loaded.")

# 2. YOLOv8 (Local Object Detector)
yolo = YOLO('yolov8n.pt')
print("✅ YOLOv8 Loaded.")

⚙️ Loading Models...


I0000 00:00:1767191038.469210     120 gpu_device.cc:2019] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 15513 MB memory:  -> device: 0, name: Tesla P100-PCIE-16GB, pci bus id: 0000:00:04.0, compute capability: 6.0


94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 5s 0us/step
✅ ResNet50 Loaded.
✅ YOLOv8 Loaded.


In [3]:
def build_sigmoid_dae(input_dim):
    """
    Sigmoid-based Autoencoder.
    Ideal for normalized features [0, 1].
    """
    inputs = layers.Input(shape=(input_dim,))
    
    # Noise Layer (Forces model to learn structure, not just copy pixels)
    x = layers.GaussianNoise(0.1)(inputs)
    
    # Encoder
    x = layers.Dense(1024, activation='sigmoid')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Dense(512, activation='sigmoid')(x)
    x = layers.BatchNormalization()(x)
    
    # Bottleneck (Compression)
    encoded = layers.Dense(32, activation='sigmoid')(x)
    
    # Decoder
    x = layers.Dense(512, activation='sigmoid')(encoded)
    x = layers.BatchNormalization()(x)
    x = layers.Dense(1024, activation='sigmoid')(x)
    x = layers.BatchNormalization()(x)
    
    # Output
    outputs = layers.Dense(input_dim, activation='sigmoid')(x)
    
    model = models.Model(inputs, outputs)
    model.compile(optimizer=optimizers.Adam(learning_rate=0.0001), loss='mse')
    return model

In [4]:
print("🚀 Phase 1: Global Stream (Scene)...")

def extract_global_features(base_dir):
    folders = sorted([f for f in glob.glob(os.path.join(base_dir, "*")) if os.path.isdir(f)])
    if not folders:
        print("❌ Error: No folders found."); return np.empty((0, 2048))
    
    all_features = []
    print(f"   Processing {len(folders)} folders...")
    
    for folder in folders:
        images = sorted(glob.glob(os.path.join(folder, "*.jpg")) + glob.glob(os.path.join(folder, "*.JPG")))
        batch = []
        for img_path in images:
            img = cv2.imread(img_path)
            if img is None: continue
            img = cv2.resize(img, IMG_SIZE)
            img = applications.resnet50.preprocess_input(img.astype('float32'))
            batch.append(img)
            
            if len(batch) >= BATCH_SIZE:
                # Use predict_on_batch for stability
                feats = resnet.predict_on_batch(np.array(batch))
                all_features.append(feats)
                batch = []
        
        if batch:
            feats = resnet.predict_on_batch(np.array(batch))
            all_features.append(feats)
        print(f"   -> Scene: {os.path.basename(folder)} done.")
            
    return np.vstack(all_features) if all_features else np.empty((0, 2048))

# Execute
global_train_data = extract_global_features(TRAIN_DIR)

🚀 Phase 1: Global Stream (Scene)...
   Processing 16 folders...


I0000 00:00:1767191059.872882     180 service.cc:152] XLA service 0x7c5b140056b0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1767191059.872924     180 service.cc:160]   StreamExecutor device (0): Tesla P100-PCIE-16GB, Compute Capability 6.0
I0000 00:00:1767191060.673310     180 cuda_dnn.cc:529] Loaded cuDNN version 91002
I0000 00:00:1767191063.356918     180 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


   -> Scene: 01 done.
   -> Scene: 02 done.
   -> Scene: 03 done.
   -> Scene: 04 done.
   -> Scene: 05 done.
   -> Scene: 06 done.
   -> Scene: 07 done.
   -> Scene: 08 done.
   -> Scene: 09 done.
   -> Scene: 10 done.
   -> Scene: 11 done.
   -> Scene: 12 done.
   -> Scene: 13 done.
   -> Scene: 14 done.
   -> Scene: 15 done.
   -> Scene: 16 done.


In [6]:
print("🚀 Phase 2: Local Stream (Objects)...")

def extract_local_features(base_dir):
    folders = sorted([f for f in glob.glob(os.path.join(base_dir, "*")) if os.path.isdir(f)])
    local_features = []
    
    for folder in folders:
        images = sorted(glob.glob(os.path.join(folder, "*.jpg")) + glob.glob(os.path.join(folder, "*.JPG")))
        
        # We accumulate CROPS, not frames, to batch them efficiently
        crop_batch = []
        
        for img_path in images:
            img = cv2.imread(img_path)
            if img is None: continue
            
            # 1. Run YOLO (Fast Mode)
            results = yolo(img, verbose=False, classes=[0]) # 0 = Person
            
            # 2. Process Detections
            for res in results:
                boxes = res.boxes.xyxy.cpu().numpy()
                for box in boxes:
                    x1, y1, x2, y2 = map(int, box)
                    
                    # Pad the crop slightly
                    h, w = img.shape[:2]
                    x1, y1 = max(0, x1-5), max(0, y1-5)
                    x2, y2 = min(w, x2+5), min(h, y2+5)
                    
                    crop = img[y1:y2, x1:x2]
                    if crop.size > 0:
                        crop = cv2.resize(crop, CROP_SIZE)
                        crop = applications.resnet50.preprocess_input(crop.astype('float32'))
                        crop_batch.append(crop)
            
            # 3. If we have enough crops, run ResNet
            if len(crop_batch) >= BATCH_SIZE:
                feats = resnet.predict_on_batch(np.array(crop_batch))
                local_features.append(feats)
                crop_batch = [] # Reset
        
        # Cleanup remaining crops
        if crop_batch:
            feats = resnet.predict_on_batch(np.array(crop_batch))
            local_features.append(feats)
            
        print(f"   -> Objects: {os.path.basename(folder)} done.")
        
    return np.vstack(local_features) if local_features else np.empty((0, 2048))

# Execute
local_train_data = extract_local_features(TRAIN_DIR)

🚀 Phase 2: Local Stream (Objects)...
   -> Objects: 01 done.
   -> Objects: 02 done.
   -> Objects: 03 done.
   -> Objects: 04 done.
   -> Objects: 05 done.
   -> Objects: 06 done.
   -> Objects: 07 done.
   -> Objects: 08 done.
   -> Objects: 09 done.
   -> Objects: 10 done.
   -> Objects: 11 done.
   -> Objects: 12 done.
   -> Objects: 13 done.
   -> Objects: 14 done.
   -> Objects: 15 done.
   -> Objects: 16 done.


In [7]:
# --- 1. Train Global DAE ---
if len(global_train_data) > 0:
    g_min, g_max = global_train_data.min(), global_train_data.max()
    global_train_data = (global_train_data - g_min) / (g_max - g_min + 1e-7)
    
    print("\n🧠 Training Global DAE...")
    global_dae = build_sigmoid_dae(2048)
    global_dae.fit(global_train_data, global_train_data, epochs=50, batch_size=128, verbose=1)
else:
    print("❌ No Global Data!")

# --- 2. Train Local DAE ---
if len(local_train_data) > 0:
    l_min, l_max = local_train_data.min(), local_train_data.max()
    local_train_data = (local_train_data - l_min) / (l_max - l_min + 1e-7)
    
    print("\n🧠 Training Local DAE...")
    local_dae = build_sigmoid_dae(2048)
    local_dae.fit(local_train_data, local_train_data, epochs=50, batch_size=128, verbose=1)
else:
    print("⚠️ No Objects detected. Local DAE will be disabled.")
    local_dae = None


🧠 Training Global DAE...
Epoch 1/50
72/72 ━━━━━━━━━━━━━━━━━━━━ 9s 36ms/step - loss: 0.2301
Epoch 2/50
72/72 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.2087
Epoch 3/50
72/72 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.1809
Epoch 4/50
72/72 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.1199
Epoch 5/50
72/72 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0608
Epoch 6/50
72/72 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0289
Epoch 7/50
72/72 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0151
Epoch 8/50
72/72 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0090
Epoch 9/50
72/72 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0059
Epoch 10/50
72/72 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0042
Epoch 11/50
72/72 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0031
Epoch 12/50
72/72 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0024
Epoch 13/50
72/72 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0020
Epoch 14/50
72/72 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0017
Epoch 15/50
72/72 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss:

In [10]:
import os
import glob
import cv2
import re  
import numpy as np
import pandas as pd
import shutil
from scipy.ndimage import gaussian_filter1d
from joblib import Parallel, delayed
from IPython.display import FileLink

# ==========================================
# 1. SAVE MODELS (Safety Step)
# ==========================================
print("💾 Saving Models...")
save_dir = 'saved_models'
if not os.path.exists(save_dir):
    os.makedirs(save_dir)

try:
    if 'global_dae' in globals(): global_dae.save(os.path.join(save_dir, 'global_dae.h5'))
    if 'local_dae' in globals(): local_dae.save(os.path.join(save_dir, 'local_dae.h5'))
    shutil.make_archive('my_models', 'zip', save_dir)
    print("✅ Models saved.")
    display(FileLink('my_models.zip'))
except Exception as e:
    print(f"⚠️ Warning: Could not save models: {e}")

# ==========================================
# 2. INFERENCE WITH ROBUST REGEX PARSING
# ==========================================
print("\n🚀 Starting Turbo Inference (Regex ID Parsing)...")

TEST_DIR = "/kaggle/input/pixel-play-26/Avenue_Corrupted-20251221T112159Z-3-001/Avenue_Corrupted/Dataset/testing_videos"
INF_BATCH = 32
results = []

# Helper function from your example code
def get_clean_number(text):
    # Finds all digit sequences and takes the last one (e.g. "Test001" -> 1)
    # int() automatically removes leading zeros (001 -> 1)
    matches = re.findall(r'\d+', text)
    if matches:
        return int(matches[-1])
    return None

test_folders = sorted([f for f in glob.glob(os.path.join(TEST_DIR, "*")) if os.path.isdir(f)])

for folder in test_folders:
    folder_name = os.path.basename(folder)
    
    # --- REGEX PARSE VIDEO ID ---
    vid_num = get_clean_number(folder_name)
    if vid_num is None: continue

    # Get Images
    images = sorted(glob.glob(os.path.join(folder, "*.jpg")) + 
                    glob.glob(os.path.join(folder, "*.JPG")) + 
                    glob.glob(os.path.join(folder, "*.tif")))
    
    print(f"   Processing Video {vid_num} ({len(images)} frames)...")

    # Batched Loop
    for i in range(0, len(images), INF_BATCH):
        batch_paths = images[i : i + INF_BATCH]
        if not batch_paths: continue

        batch_frames = []
        batch_ids = []
        
        for path in batch_paths:
            img = cv2.imread(path)
            if img is not None:
                img_resized = cv2.resize(img, (224, 224))
                batch_frames.append(img_resized)
                
                # --- REGEX PARSE FRAME ID ---
                # Extracts '00939' from 'frame_00939.tif' and converts to 939
                fname = os.path.basename(path)
                frame_num = get_clean_number(fname)
                
                # Create Strict ID: "1_939"
                batch_ids.append(f"{vid_num}_{frame_num}")

        if not batch_frames: continue
        
        # Predict (Your Model Logic)
        g_batch = np.array(batch_frames).astype('float32')
        g_batch = applications.resnet50.preprocess_input(g_batch)

        g_feats = resnet.predict_on_batch(g_batch)
        g_feats_norm = (g_feats - g_min) / (g_max - g_min + 1e-7)
        g_recons = global_dae.predict_on_batch(g_feats_norm)
        g_mses = np.mean(np.square(g_feats_norm - g_recons), axis=1)

        # Store
        for j in range(len(batch_frames)):
            results.append({
                'Id': batch_ids[j],      # e.g., "1_939"
                'Video': vid_num,
                'Raw_Error': g_mses[j]
            })

# ==========================================
# 3. SMOOTHING & SAVE
# ==========================================
print("⚡ Smoothing & Formatting...")
df = pd.DataFrame(results)

# Calc Scores
max_err = df['Raw_Error'].max()
df['PSNR'] = 10 * np.log10((max_err**2) / (df['Raw_Error'] + 1e-10))
p_min, p_max = df['PSNR'].min(), df['PSNR'].max()
df['Anomaly_Score'] = 1 - ((df['PSNR'] - p_min) / (p_max - p_min + 1e-10))

# Parallel Smoothing
def smooth_chunk(idx, scores):
    return idx, gaussian_filter1d(scores, sigma=4)

smoothed_results = Parallel(n_jobs=-1)(
    delayed(smooth_chunk)(grp.index, grp['Anomaly_Score'].values)
    for _, grp in df.groupby('Video')
)

df['Smoothed'] = 0.0
for idx, val in smoothed_results:
    df.loc[idx, 'Smoothed'] = val

# Save Final
df[['Id', 'Smoothed']].rename(columns={'Smoothed': 'Predicted'}).to_csv('submission.csv', index=False)

print(f"✅ DONE! Saved 'submission.csv'")
print(f"   Sample ID Check: {df.iloc[-1]['Id']} (Should be formatted like '21_120' etc.)")

💾 Saving Models...
✅ Models saved.


/kaggle/working/my_models.zip


🚀 Starting Turbo Inference (Regex ID Parsing)...
   Processing Video 1 (499 frames)...
   Processing Video 2 (1211 frames)...
   Processing Video 3 (737 frames)...
   Processing Video 4 (947 frames)...
   Processing Video 5 (1007 frames)...
   Processing Video 6 (627 frames)...
   Processing Video 7 (588 frames)...
   Processing Video 8 (36 frames)...
   Processing Video 9 (359 frames)...
   Processing Video 10 (722 frames)...
   Processing Video 11 (472 frames)...
   Processing Video 12 (735 frames)...
   Processing Video 13 (528 frames)...
   Processing Video 14 (496 frames)...
   Processing Video 15 (732 frames)...
   Processing Video 16 (740 frames)...
   Processing Video 17 (417 frames)...
   Processing Video 18 (275 frames)...
   Processing Video 19 (229 frames)...
   Processing Video 20 (273 frames)...
   Processing Video 21 (76 frames)...
⚡ Smoothing & Formatting...
✅ DONE! Saved 'submission.csv'
   Sample ID Check: 21_75 (Should be formatted like '21_120' etc.)


In [1]:
import pandas as pd
import numpy as np
from scipy.ndimage import gaussian_filter1d

# 1. Load your best baseline
df = pd.read_csv('/kaggle/input/res-yo/submission (1).csv')
df[['Video', 'Frame']] = df['Id'].str.split('_', expand=True).astype(int)
df = df.sort_values(['Video', 'Frame'])

# 2. MSE -> PSNR Conversion
# Peak Signal-to-Noise Ratio focuses on the reconstruction quality
epsilon = 1e-10
df['PSNR'] = 10 * np.log10(1.0 / (df['Predicted'] + epsilon))

# 3. Choose your Normalization Strategy
# If Per-Video lowered your score before, try 'GLOBAL'
STRATEGY = 'PER_VIDEO' 

if STRATEGY == 'PER_VIDEO':
    def norm_func(x):
        return 1 - ((x - x.min()) / (x.max() - x.min() + epsilon))
    df['Anomaly_Score'] = df.groupby('Video')['PSNR'].transform(norm_func)
else:
    # Global normalization preserves the 'intensity' difference between videos
    p_min, p_max = df['PSNR'].min(), df['PSNR'].max()
    df['Anomaly_Score'] = 1 - ((df['PSNR'] - p_min) / (p_max - p_min + epsilon))

# 4. Gaussian Smoothing (More precise than Moving Average)
# sigma=1.0 or 1.5 is the 'sweet spot' for 24-30fps video
df['Smoothed'] = df.groupby('Video')['Anomaly_Score'].transform(
    lambda x: gaussian_filter1d(x, sigma=1.2)
)

# 5. Save the two best candidates
df[['Id', 'Anomaly_Score']].rename(columns={'Anomaly_Score': 'Predicted'}).to_csv('sub_psnr_only.csv', index=False)
df[['Id', 'Smoothed']].rename(columns={'Smoothed': 'Predicted'}).to_csv('sub_psnr_gaussian.csv', index=False)

print("Generated 'sub_psnr_only.csv' and 'sub_psnr_gaussian.csv'")

Generated 'sub_psnr_only.csv' and 'sub_psnr_gaussian.csv'
